# Decision Optimization Notebook - Retail SC Optimization 

This is an example of a 'traditional' notebook approach to implement an optimization model.<br>

Stand-alone/traditional in the sense:
* This notebook can be used as a solver notebook in DO4WS.
* Uses a few stand-alone re-usable functions defined at the start of the notebook. Includes only those functions it needs.
* All inputs, outputs and intermediate data DataFrames are defined globally.
* No object-oriented code.
* No re-use and sharing of code (no import from custom modules or packages).
* No use of `dse-do-utils`

# READ ME<a class="anchor" id="Read_Me"></a>
This notebook solves a simplified production planning model that is part of a larger supply chain optimization problem.

The supply-chain model we'll be building is a simplified version that only considers production facilities and warehouses (Suppliers, Manufacturers, Warehouse) and demand, i.e.:
* Multiple plants, from multiple suppliers in multiple locations.
* Same line may be capable of producing a range of products, each which their own production rate and cost
* Multiple products, where different suppliers compete, i.e. are capable of producing the same product.
* Multiple warehouses and different warehouse locations
* Warehouses have demand for the various products at various locations

Simplification:
* PLEASE NOTE: In order to set a flexible foundation for various client environments, simplification must be made in lieu of company specific data. Any of the following simplifications can be resolved with actual or estimated historical cost/demand/route data.
* Scenario data including routes, estimated costs, current inventory has been affixed to the LSTM forecasted demand and saved to All_Routes_Table.csv
* Transportation cost is a hardcoded linear cost pending straight line point to point travel distance (Google API can be substituted to find route distance by roadmap)
* Transportation time is the summation of number of stops after the supplier. A route passing from one supplier to one manufacturer to one warehouse will be 2 days
* Demand forecast has already been generated by LSTM model (or other user defined model) and saved to data assets
* Optimization model will solve for the first demand period generated by the forecast
* Each supplier/manufacturer combination can produce product in demand
* No inventory
* User has enabled the decision optimization cartridge on Cloud Pak for Data 3.0+

Supply Chain optimization will plan:
* Production activities: how much of which product to produce on which line for which warehouse.
* Backlog: Whatever demand cannot is not met will be stored into an order backlog

Such that:
* Minimal production cost
* Minimal travel cost
* Minimal backlog cost
* Meet demand
* Below or equal to supplier capacity

The input tables for this model:
* Product
* Location
* Plant
* Line
* Demand

# Package Management<a class="anchor" id="package_management"></a>
#### Use the following code block to install any missing packages

#### Import packages and print environment health check

In [ ]:
import pandas as pd
import numpy as np
from docplex.mp.environment import Environment
from docplex.mp.model import Model
env = Environment()
env.print_information()

#### Model Definition

In [ ]:
mdl = Model(name="ProductionPlanning")

# Read Data Inputs<a class="anchor" id="read_data"></a>

#### Pull in the demand data

In [ ]:
#Demand
SC_data = inputs['DO_Route_Data']
SC_data.head()

#### Pull in the demand data
This dataset provides the forecasted demand for every warehouse product combination. 

In [ ]:
demand_df = SC_data[['Destination','Product_Code','Forecast']].drop_duplicates().reset_index(drop = True)
demand_df.head()

#### Pull in our backlog data (currently inventory)
This csv file contains the backlog for every product in every warehouse.

In [ ]:
#Backlog
backlog_df = inputs['Backlog']
backlog_demand = backlog_df.merge(demand_df, left_on = ('Destination', 'Product_Code'), right_on= ('Destination', 'Product_Code'))
backlog_demand

# Create decision variables<a class="anchor" id="decision_variables"></a>
#### Re-usable functions to emphasise best-practice pattern.
#### Added as new method on a CPLEX Model so usage is very close to a call to the other methods of Model.

In [ ]:
from docplex.mp.model import Model
def integer_var_series(self, df, **kargs):
    '''Create a Series of integer dvar for each row in the DF. Most effective method. Best practice.
    Result can be assigned to a column of the df.
    Usage:
        df['xDVar'] = mdl.integer_var_series(df, name = 'xDVar')
    Args:
        mdl (docplex.mp.model): CPLEX Model
        df (DataFrame): dataframe
        **kargs: arguments passed to mdl.integer_var_list method. E.g. 'name'
    Returns:
        (pandas.Series) with integer dvars, index matches index of df
    '''
    #We are re-using the index from the DF index:
    return pd.Series(self.integer_var_list(df.index, **kargs), index = df.index)
def continuous_var_series(self, df, **kargs):
    return pd.Series(self.continuous_var_list(df.index, **kargs), index = df.index)
def binary_var_series(self, df, **kargs):
    return pd.Series(self.binary_var_list(df.index, **kargs), index = df.index)
#Bind method to class:
Model.integer_var_series = integer_var_series
Model.continuous_var_series = continuous_var_series
Model.binary_var_series = binary_var_series

### Data Checker<a class="anchor" id="data_checker"></a>
For now assume that input data originates from a relational database that ensures:
* All the required columns are present
* The names of the columns are as expected
* Each column has the expected data type
* Null vs None-null values as expected
* Relational integrity: foreign keys exist as rows in their source table

### Pre-processing<a class="anchor" id="pre_procesing"></a>
The basic building blocks for this optimization problem are the `ProductionActivities` and `AggregateDemand`, i.e. what Product to produce on what Line:
* (key) Product
* (key) Line
* quantity

Steps:
1. Get active active Demand and active Products
2. Create ProductionActivities that can manufacture a Product on a Production Line in a TimePeriod
3. Aggregate Demand

### ProductionActivities 
1. Create all combinations between a Product and Product Line.
2. Create all combinations between a Product, Warehouse and Production Line.

Production Activities are the basis for the production quantity decision variables.

### Aggregate Demand
Aggregate Demand is the basis for the demand fulfillment constraints and the backlog decision variables.<br>
We aggregate demand over each product, so we can define a backlog per product.<br>
In this simplified demo, we do not yet model backlog per time-period, which requires a backlog-resupply.

# Objective Function Definition<a class="anchor" id="objective"></a>

In [ ]:
# Create productionactivities and aggregatedemand tables for clarity
pa = SC_data[SC_data['Product_Code'].notna()]
ad = backlog_demand
backlog_penalty = 10 #For now, hard-code the back log penalty
pa.Miles = pa.Miles/100000 #For now, hard-code cost. Drive down milage to create simplified cost per product

name_f = tuple(list(pa.prodx))
pa['xProd_vecm'] = mdl.integer_var_series(pa, lb = 0, name = name_f)

#### Minimize cost:
-  Production cost:
    -  _sum(pa) xProd[pa] * pa.costPerPart_
    -  i.e. the production quantity * the production cost per part
-  Backlog cost: 
    -  _sum(ad) xBacklog[ad] * backlogPenalty_
    -  i.e. the backlog quantity * backlog penalty. For now the backlog penalty is a static number (100)
    
The backlog costs are set an order of magnitude higher than the production cost. This will force the model to make backlog reduction the highest priority, before minimizing the production cost.<br>
_Note: an alternative approach is to define a lexicographical/hierarchical optimization approach._

#### Production Cost: 
The production cost is the amount of units supplied per given product route multiplied by the production cost.

$$
productionCost = \sum\limits_{\forall ProductionActivity} xProd[pa] * pa.costPerPart
$$

In [ ]:
production_cost = mdl.sum(pa.xProd_vecm * pa.cost) #Uses vector multiplication, very efficient
mdl.add_kpi(production_cost, "Production Cost")

#### Backlog Cost: 
Backlog cost is the number of products that can't immediately be created due to production capacity constraints for each warehouse multiplied by the penalty assigned for each backlog unit. 
$$
backlogCost = \sum\limits_{\forall AggregatedDemand} xBacklog[ad] * penalty
$$

In [ ]:
ad['xBacklog'] = ad['Destination'] + ad['Product_Code']
backlog_f = tuple(list(ad.xBacklog))
ad['xBacklog_vecm'] = mdl.integer_var_series(ad, lb = 0, name = backlog_f)

In [ ]:
backlog_cost = mdl.sum(ad.xBacklog_vecm * backlog_penalty)
mdl.add_kpi(backlog_cost, "Backlog Cost")

#### Travel Cost: 
Travel cost is the number of units supplied for each unique product route multiplied by the distance (as measured in miles).
$$
TravelCost = \sum\limits_{\forall ProductionActivity} xProd[pa] * xMile
$$

In [ ]:
travel_cost = mdl.sum(pa.xProd_vecm * pa.Miles)
mdl.add_kpi(travel_cost, "Travel Cost")

#### Objective Function
In this use case, our objective function attempts to minimize the backlog, production and travel costs. The cell below takes the various costs that we have created above and defines our objective function. 

In [ ]:
mdl.minimize(backlog_cost + production_cost + travel_cost)

# Constraint Definition<a class="anchor" id="constraint"></a>
-  Production capacity:
    -  _For each line l:_ 
        -  _sum(pa in l) xProd[pa] / lineCapacity[l,pa.p] <= 1_
    -  Each line has a different capacity to produce a product. Therefore, we compute an expression for the line utilization per pa. The total utilization must be below 100%
-  Fulfill demand:
    -  _For each aggregated demand product p:_ 
        -  _sum(pa in p) xProd[pa] >= p.aggregatedDemand - xBacklog[p]_
    -  The total production for a part needs to be larger than the total demand, minus the backlog

In [ ]:
ad.head()

In [ ]:
#create forecast compensating for backlog
ad['forecast_backlog'] = ad['Forecast'] - ad['backlog']

#### Demand Fulfillment Constraint: 
This constraint stipulates that for every product and its final warehouse destination the quantity supplied is always greater than or equal to the the forecast minus the original backlog as well as any additional backlog added. 

In [ ]:
#Demand Fulfillment Constraint
for (product, warehouse), group in pa.groupby(['Product_Code', 'Destination']):
    ad1 = ad[(ad['Product_Code']==product) & (ad['Destination']==warehouse)].sum()
    mdl.add_constraint(
        mdl.sum(group.xProd_vecm) 
         >= 
        ad1.forecast_backlog - ad1.xBacklog_vecm, 
         'DemandFulfillmentC_{}'.format(product)
    )

#### Capacity Constraint: 
This constraint stipulates that the amount of quantity supplied for every product supplier combination 
does not exceed the amount that the given supplier can create for a given product. 


In [ ]:
#Capacity Constraint
for (product, supplier), group in pa.groupby(['Product_Code', 'SupplierID']):
    mdl.add_constraint(
        (mdl.sum(mdl.sum(group.xProd_vecm * (1/group.capacity)) <= 1)), 'WarehouseCapacityC_{}_{}'.format(product, supplier))

# Run CPLEX Model<a class="anchor" id="run_model"></a>
#### Now that our model is defined with an objective function and constraints, we can solve!

In [ ]:
# CPLEX:
# Additionally you can add a time parameter to the solve method below: cplex_parameters = {'timelimit':TIME_LIMIT})
msol = mdl.solve(log_output=True)
if msol is not None:
    mdl.report()

#### Conflict resolution<a class="anchor" id="conflict_resolution"></a>
For larger sized decision models, it can be hard to find what is causing the model solve to fail. This is where the CPLEX conflict refiner can help. If the model is infeasible, the conflict refiner will search for the minimal conflict. The minimal conflict is the smallest set of constraints and variables that are in conflict with each other, i.e. where it is impossible to find a feasible solution among the decision variables that satisfy all constraints in the set.

Knowing the conflict can help understanding the modelling error and thereby help debugging the model generation code.

For details, see the reference manual: http://ibmdecisionoptimization.github.io/docplex-doc/mp/docplex.mp.conflict_refiner.html

Run the CPLEX conflict resolution to find out what constraints are conflicting:

In [ ]:
from docplex.mp.conflict_refiner import ConflictRefiner
if msol is None:
    crefiner = ConflictRefiner() # Create an instance of the ConflictRefiner
    conflicts = crefiner.refine_conflict(mdl) # Run the conflict refiner
    for c in conflicts:
        print(c.element) #Display conflict result in a little more compact format than ConflictRefiner.display_conflicts

# Post Processing<a class="anchor" id="post"></a>

#### The cell below provides the total production, backlog, and travel costs - respectively.

In [ ]:
all_kpis = [(kp.name, kp.compute()) for kp in mdl.iter_kpis()]
df_kpis = pd.DataFrame(all_kpis, columns=['kpi', 'value'])
df_kpis

#### Extract solution
-  ProductionActivity
-  Backlog

*Note: The reason we explicitly extract the solution in a separate column, versus indirectly referring to the value using the `.solution_value` attribute of the dvar is that the column value will allow us to do vector operations to compute derived values.*

In [ ]:
pa['production_quantity'] = [dvar.solution_value for dvar in pa.xProd_vecm]
ad['backlog_quantity'] = [dvar.solution_value for dvar in ad.xBacklog_vecm]
pa_nozeros = pa[pa['production_quantity']>0]

product_category = inputs['Product_Category_Legend']
product_category.columns = ['Index', 'Product_Category', 'Product_Code', 'Total']
del product_category['Index']
del product_category['Total']

location_df = inputs['Locations']
location_df=location_df.rename(columns = {'Warehouse':'Warehouse_City'})

In [ ]:
pa_withproduct = pa_nozeros.merge(product_category,how='left',left_on='Product_Code',right_on='Product_Code')
pa_prodandlocation = pa_withproduct.merge(location_df,how='left',left_on='Destination',right_on='WarehouseID')

In [ ]:
outflow = pa_prodandlocation[['Product_Code','Warehouse', 'production_quantity', 'cost']]
outflow = outflow.groupby(['Product_Code', 'Warehouse'], as_index = False).agg({'production_quantity' : 'sum', 'cost' : 'sum'})
forecast = inputs['Transposed_Forecast_Table']
Outflow_Demand = outflow.merge(forecast, left_on = ('Product_Code', 'Warehouse'), right_on= ('Product', 'WarehouseID'))
Outflow_Demand = Outflow_Demand.merge(product_category,how='left',left_on='Product_Code',right_on='Product_Code')

In [ ]:
Outflow_Demand=Outflow_Demand.rename(columns = {'Quantity':'Forecast_Quantity', 
                                                'production_quantity':'Production_Quantity',
                                                'cost':'Unit Cost'})
Outflow_Demand['Forecast_Quantity'][Outflow_Demand['Forecast_Quantity'] == 0] = 1
Outflow_Demand['Pct_Satisfied'] = Outflow_Demand['Production_Quantity'] / Outflow_Demand['Forecast_Quantity']
Outflow_Demand.head()

In [ ]:
outputs['Solution_Table'] = pa_prodandlocation

In [ ]:
outputs['Outflow_Demand'] = Outflow_Demand

**Sample Materials, provided under license. <br>
Licensed Materials - Property of IBM. <br>
© Copyright IBM Corp. 2020 <br>
All Rights Reserved. <br>
US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with IBM Corp.**